In [3]:
import numpy as np
import pandas as pd

from scipy.stats import norm
from scipy.optimize import minimize
from math import ceil

import warnings
warnings.filterwarnings('ignore')

In [4]:
class Option():
    def __init__(self, price, strike, sigma , start, end, call = True, EU = True, riskfree=0.075, divs = True, n=10):
        
        self.asset_price = price
        
        self.strike = strike
        
        self.sigma = sigma
        
        self.EU = EU
        
        self.riskfree = riskfree
        
        self.__call = call
        
        self.__n = n 
        
        self.__divs = divs 
        
        self.start = pd.to_datetime(start, dayfirst=True)
        self.end = pd.to_datetime(end, dayfirst=True)
        self.days = (self.end - self.start).days
        self.T = self.days/365
        self.dt = self.T/n
    

    
    def grow_tree(self, n_steps = None):
        
        """"
        
        Creates binomial tree in matrix form: from initial value - step to the RIGHT = step up; step DOWN = step down
        n_steps: number of steps for a tree
        opt_price: if False, returns tree only for the asset price; if True returns asset price tree AND option price tree
        
        """   
        if n_steps == None:
            n_steps = self.__n
        
        self.up = np.exp(self.sigma*np.sqrt(self.dt))
        self.down = 1/self.up 
        
        self.growth_factor = 1 if self.__divs == False else np.exp(self.riskfree*self.dt)
        self.p = (self.growth_factor-self.down)/(self.up-self.down)
        self.q = 1 - self.p
        
        ###########  Ценовое дерево ############

        first_row = [self.asset_price]

        for i in range(n_steps):
            first_row.append(first_row[-1]*self.up)

        tree = np.array(first_row).reshape(1, n_steps+1)

        for i in range(1, n_steps+1):
            temp = np.append(tree[-1, :-i]*self.down, [np.NaN]*i)
            tree = np.vstack([tree, temp])

        self.tree = pd.DataFrame(tree)

        price_diff = self.tree - self.strike if self.__call == True else self.strike - self.tree
        price_diff = np.where(price_diff > 0, price_diff, 0)
        
        ###########  Опционное дерево ############
            
        last_col = pd.DataFrame(np.diagonal(np.fliplr(price_diff)))

        for i in range(1, len(last_col)):
            last_col[i] = (last_col.iloc[:, -1]*self.p + last_col.iloc[:, -1].shift(periods=-1)*self.q)*np.exp(-self.riskfree*self.dt)
            
            if self.EU != True:
                temp = np.diagonal(np.fliplr(price_diff), i)
                temp = np.append(temp, [np.NaN]*i)
                last_col[i] = np.where(last_col[i] >= temp, last_col[i], temp) 

        last_col = last_col.iloc[:, ::-1]
        
        for i in last_col.index:
            last_col.loc[i, :] = last_col.loc[i, :].shift(periods=-i)
            
        last_col.columns = range(len(last_col))
        self.tree_opt = last_col
        
        self.pretty_tree = self.prettify(self.tree)
        self.pretty_opt = self.prettify(self.tree_opt)
        return self
    
    
    def prettify(self, tree):
        
        """"
        
        Transforms tree into classical form
        
        """       
        
        pretty_tree = pd.DataFrame(columns = tree.columns)
        
        for i in tree.index:
            pretty_tree.loc[i, :] = tree.loc[i, :].shift(periods=i)
            pretty_tree.loc[i+0.5, :] = np.NaN

        pretty_tree = pretty_tree.sort_index().reset_index(drop=True)

        for i, column in enumerate(pretty_tree.columns):
            pretty_tree[column] = pretty_tree[column].shift(periods=(int(len(pretty_tree.index)/2)-i))
        
        pretty_tree = pretty_tree.fillna('')
        
        return pretty_tree
    
    
    def combine(self):
        
        """"
        
        Combines option and asset tree into one
        
        """
        
        opt = self.prettify(self.tree_opt)
        opt.index += 1
        opt.loc[0, :] = 0

        asset = self.prettify(self.tree)
        asset.loc[len(asset), :] = 0
        self.tree_comb = (opt.replace('', 10**(-100)) + asset.replace('', 0)).replace(10**(-100), '').drop(0).reset_index(drop=True)
        
        return self

    
    
    
    def BSM(self, precise = False):
        
        """"
        
        Calculate option's price according to Blasck-Scholes-Merton model
        
        precise: defines the value of n, if False: it uses value given during initialization, otherwise, it it equal to 1 mln
        
        """
        
        if self.EU == False:
            raise Exception('American options cannot be evaluated using Black-Scholes model')
        
        dt_precise = 0 if precise != False else self.dt
        
        self.d1 = (np.log(self.asset_price/self.strike) + (self.riskfree + self.sigma**2/2)*(self.T - dt_precise))/(self.sigma*np.sqrt(self.T - dt_precise))
        
        self.d2 = self.d1 - self.sigma*np.sqrt(self.T - dt_precise)
        
        if self.__call == True:
            self.BSM_price = norm.cdf(self.d1)*self.asset_price - norm.cdf(self.d2)*self.strike*np.exp(-1*self.riskfree*(self.T-dt_precise))
        else:
            self.BSM_price = norm.cdf(-self.d2)*self.strike*np.exp(-1*self.riskfree*(self.T-dt_precise)) - norm.cdf(-self.d1)*self.asset_price 

        return self.BSM_price
    
    
    
    def get_net(self, m = 30, width = None):
        
        if width == None:
            width = self.__n
        net = pd.DataFrame()
        
        max_col = list(self.tree_opt.iloc[0, :]).index(0) #if self.__call == False else list(self.tree_opt.iloc[:, 0]).index(0)
        s_max = self.tree.iloc[0, max_col]
    
        self.dS = self.asset_price/m
        k = ceil(s_max/self.dS)
        j = np.arange(k+1)[::-1]

        ###########  Columns ############

        first_col = []
        for i in j:
            first_col.append(i*self.dS)
        
        first_col = np.array(first_col)
        
        # price_diff = first_col - self.strike if self.__call == True else self.strike - first_col ####????????
        
        price_diff = self.strike - first_col    

        last_col = np.where(price_diff>0, price_diff, 0) 


        ###########  Rows ############

        first_row = ['Share Price/ Time']
        for i in range(width+1):
            time = round(i*self.T/width, 5)
            first_row.append(time)
        
        
        ###########  ABC ############
        
        div = 0 if self.__divs==False else float(input('enter dividend yield: '))
        
        abc = [0, 1 , 0]

        for num in j[1:-1]:    
            aj = 0.5*(self.riskfree-div)*num*self.dt - 0.5*self.sigma**2*num**2*self.dt

            bj = 1 + self.sigma**2*num**2*self.dt + self.riskfree*self.dt

            cj = -0.5*(self.riskfree-div)*num*self.dt - 0.5*self.sigma**2*num**2*self.dt

            abc = np.vstack([abc, [aj, bj, cj]])

        self.abc = np.vstack([abc, [0, 1, 0]]).T

        zeroes = np.zeros([len(last_col) - self.abc.shape[0], self.abc.shape[1]])
        abc_matrix = np.vstack([self.abc, zeroes])

        for i in range(len(abc_matrix)):
            abc_matrix[:, i] = np.roll(abc_matrix[:, i], i-1)

        inv = np.linalg.inv(abc_matrix)
        
        ###########  Grid ############
            
        net = pd.DataFrame()
        net[0] = last_col

        for i in range(1, width+1):
            net[i] = net.iloc[:, -1] @ inv
            
            if self.EU != True:
                net[i] = np.where(net[i]>last_col, net[i], last_col)
        
        net['Share price'] = first_col
        net = net.iloc[:, ::-1]
        net.columns = first_row
        
        return net
        
    def params(self):
        return self.__dict__

In [25]:
yndx = Option(473.4, 540, 0.4676, start = '26/06/2023', end = '16/08/2023', EU = 0, call = 0, n=51)

In [26]:
yndx.BSM(True)

Exception: American options cannot be evaluated using Black-Scholes model

In [27]:
yndx.grow_tree()

In [28]:
yndx.tree_opt

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,75.023465,66.317994,57.981019,50.079933,42.679357,35.838276,29.606782,24.022785,19.109143,14.871558,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,83.693022,74.618312,65.845369,57.444371,49.485550,42.036748,35.160162,28.908653,23.322118,18.424355,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,92.732541,83.354660,74.209025,65.365796,56.897601,48.877913,41.378218,34.464266,28.191943,22.603328,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
3,102.076613,92.464422,83.014953,73.795939,64.879203,56.340218,48.256194,40.702691,33.749327,27.455291,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
4,111.657026,101.879424,92.197394,82.674643,73.379394,64.385436,55.771640,47.619499,40.009008,33.013912,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
5,121.405507,111.529061,101.685008,91.932597,82.334557,72.959664,63.884220,55.191238,46.966945,39.295839,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
6,131.256066,121.343432,111.404408,101.494815,91.671405,81.995518,72.536823,63.375181,54.598566,46.297514,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
7,141.138763,131.256066,121.283767,111.284592,101.310767,91.415387,81.658042,72.110568,62.858438,53.993131,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,150.782514,141.138763,131.256066,121.227643,111.171888,101.135479,91.165874,81.321409,71.681626,62.334210,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,160.193097,150.782514,141.138763,131.256066,121.176997,111.070185,100.972318,90.920705,80.986847,71.251093,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
yndx.get_net().head(50)

enter dividend yield: 0


,Share Price/ Time,0.0,0.00274,0.00548,0.00822,0.01096,0.0137,0.01644,0.01918,0.02192,...,0.11507,0.11781,0.12055,0.12329,0.12603,0.12877,0.13151,0.13425,0.13699,0.13973
0,978.36,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00
1,962.58,0.013353,0.012010,0.010765,0.009613,0.008550,0.007575,0.006681,0.005867,0.005127,...,3.558937e-08,1.410416e-08,5.071810e-09,1.622646e-09,4.494984e-10,1.036107e-10,1.865125e-11,2.333452e-12,1.523645e-13,0.00
2,946.80,0.027756,0.025000,0.022439,0.020067,0.017878,0.015863,0.014016,0.012329,0.010794,...,8.995708e-08,3.611215e-08,1.316529e-08,4.274164e-09,1.202665e-09,2.818854e-10,5.165665e-11,6.587277e-12,4.389978e-13,0.00
3,931.02,0.044544,0.040208,0.036172,0.032426,0.028960,0.025763,0.022826,0.020135,0.017681,...,1.933762e-07,7.903693e-08,2.936810e-08,9.728693e-09,2.796582e-09,6.704892e-10,1.258567e-10,1.646355e-11,1.127272e-12,0.00
4,915.24,0.065280,0.059097,0.053325,0.047953,0.042968,0.038356,0.034103,0.030195,0.026618,...,4.046680e-07,1.688436e-07,6.411688e-08,2.173232e-08,6.399998e-09,1.574077e-09,3.035377e-10,4.085307e-11,2.882742e-12,0.00
5,899.46,0.091886,0.083470,0.075588,0.068226,0.061370,0.055003,0.049110,0.043673,0.038674,...,8.462301e-07,3.609558e-07,1.402882e-07,4.872662e-08,1.472377e-08,3.720918e-09,7.383651e-10,1.024270e-10,7.462371e-12,0.00
6,883.68,0.126789,0.115619,0.105119,0.095275,0.086068,0.077484,0.069502,0.062104,0.055270,...,1.778372e-06,7.762470e-07,3.091035e-07,1.101405e-07,3.418994e-08,8.889396e-09,1.817726e-09,2.602877e-10,1.961124e-11,0.00
7,867.90,0.173115,0.158513,0.144732,0.131756,0.119569,0.108153,0.097488,0.087556,0.078333,...,3.759947e-06,1.680956e-06,6.864626e-07,2.511975e-07,8.019822e-08,2.147974e-08,4.532338e-09,6.709538e-10,5.236802e-11,0.00
8,852.12,0.234921,0.216026,0.198118,0.181182,0.165202,0.150163,0.136045,0.122827,0.110490,...,7.998151e-06,3.665644e-06,1.536720e-06,5.781235e-07,1.900598e-07,5.250765e-08,1.144969e-08,1.755178e-09,1.421705e-10,0.00
9,836.34,0.317499,0.293239,0.270145,0.248206,0.227410,0.207741,0.189183,0.171719,0.155328,...,1.711391e-05,8.048257e-06,3.467221e-06,1.342566e-06,4.550750e-07,1.298687e-07,2.931207e-08,4.661280e-09,3.926233e-10,0.00


# dfghjk

In [13]:
yndx.pretty_opt

,0,1,2,3,4,5,6,7,8,9,10
0,,,,,,,,,,,
1,,,,,,,,,,,0.0
2,,,,,,,,,,0.0,
3,,,,,,,,,0.0,,0.0
4,,,,,,,,0.0,,0.0,
5,,,,,,,0.725771,,0.0,,0.0
6,,,,,,3.958504,,1.440575,,0.0,
7,,,,,10.990405,,7.143895,,2.85938,,0.0
8,,,,22.118348,,17.924273,,12.764024,,5.67555,
9,,,36.855757,,33.10098,,28.556596,,22.524939,,11.265335


In [11]:
exam = Option(46672, 50000, 0.255, start = '29/06/2023', end = '19/07/2023', EU = True, call = False, n=50)

In [12]:
exam.BSM()

3323.437884670835

In [13]:
a = pd.Series(exam.params())

In [14]:
a = a.drop(['_Option__call', '_Option__n', '_Option__divs'])

In [15]:
a['n'] = 50
a['Type'] = 'Put'
a['Dividend yield'] = 0.0792

In [16]:
a

asset_price                     46672
strike                          50000
sigma                           0.255
EU                               True
riskfree                        0.075
start             2023-06-29 00:00:00
end               2023-07-19 00:00:00
days                               20
T                            0.054795
dt                           0.001096
d1                          -1.067933
d2                          -1.127024
BSM_price                 3323.437885
n                                  50
Type                              Put
Dividend yield                 0.0792
dtype: object

In [17]:
exam.grow_tree()

In [21]:
exam.grow_tree().get_net()

enter dividend yield:  0.12


,Share Price/ Time,0.0,0.0011,0.00219,0.00329,0.00438,0.00548,0.00658,0.00767,0.00877,...,0.04493,0.04603,0.04712,0.04822,0.04932,0.05041,0.05151,0.0526,0.0537,0.05479
0,60673.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,59117.866667,3.550436,3.298121,3.057348,2.827981,2.609874,2.402862,2.206768,2.021398,1.846545,...,0.002324,0.001392,0.000786,0.000412,0.000196,0.000082,0.000029,0.000008,0.000001,0.000000
2,57562.133333,12.233426,11.490538,10.773835,10.083332,9.419016,8.780845,8.168748,7.582622,7.022332,...,0.023186,0.014841,0.009004,0.005108,0.002656,0.001227,0.000478,0.000142,0.000025,0.000000
3,56006.400000,36.788806,34.988305,33.228935,31.511437,29.836536,28.204942,26.617343,25.074405,23.576765,...,0.219229,0.150771,0.098954,0.061218,0.035057,0.018062,0.007973,0.002736,0.000563,0.000000
4,54450.666667,102.129346,98.318328,94.547682,90.819233,87.134864,83.496512,79.906175,76.365905,72.877808,...,1.934938,1.436434,1.025830,0.697295,0.444055,0.258350,0.131407,0.053419,0.013558,0.000000
5,52894.933333,259.378085,252.524502,245.667908,238.809938,231.952354,225.097050,218.246066,211.401594,204.565992,...,15.375250,12.373874,9.671006,7.279431,5.210936,3.475917,2.082895,1.037942,0.343988,0.000000
6,51339.200000,593.542512,583.580591,573.529943,563.388894,553.155750,542.828799,532.406317,521.886572,511.267831,...,103.788779,90.776448,77.965766,65.413609,53.185656,41.357662,30.016899,19.263819,9.213939,0.000000
7,49783.466667,1205.548808,1194.659512,1183.635979,1172.473621,1161.167582,1149.712718,1138.103576,1126.334364,1114.398927,...,538.756784,509.968178,479.783426,448.073740,414.695981,379.490925,342.281302,302.869593,261.035538,216.533333
8,48227.733333,2152.441492,2144.319296,2136.143574,2127.914157,2119.630943,2111.293912,2102.903136,2094.458788,2085.961162,...,1800.683606,1794.466844,1788.790209,1783.732724,1779.383525,1775.843148,1773.224987,1771.656941,1771.283263,1772.266667
9,46672.000000,3390.487117,3386.837350,3383.223418,3379.647681,3376.112619,3372.620838,3369.175076,3365.778207,3362.433248,...,3310.367387,3311.603315,3313.044830,3314.686377,3316.519592,3318.532788,3320.710374,3323.032172,3325.472645,3328.000000


In [78]:
eu.to_excel('eu_net.xlsx')